<H3> 1) What you need before starting</H3>
Software<br>
Neo4j (running locally or via Aura)<br>
Jupyter Notebook<br>
Python 3.9+<br>

In [1]:
#2) Install required Python packages
#!pip install neo4j pandas

<H3> 3) Start Neo4j and get connection details</H3>

From Neo4j Desktop or Neo4j Aura, note:<br>

URI:<br>
Local: neo4j://127.0.0.1:7687<br>
Username: usually neo4j<br>
Password: password<br>


In [2]:
#4) Connect to Neo4j from Jupyter
from neo4j import GraphDatabase

URI = "neo4j://127.0.0.1:7687"
USERNAME = "neo4j"
PASSWORD = "password"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

print("✅ Connected to Neo4j successfully")


✅ Connected to Neo4j successfully


In [4]:
#5) Helper function to run Cypher safely
def run_cypher(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

In [5]:
# 6) Clear the database (optional but recommended for learning)
run_cypher("MATCH (n) DETACH DELETE n")
print("Database cleared")

Database cleared


<H3 >7) Create the Malaria Knowledge Graph </H3>

In [6]:
# 7.1) Disease and Parasite
query = """
MERGE (m:Disease {name:"Malaria", category:"Infectious"})
MERGE (p:Parasite {name:"Plasmodium falciparum", severity:"High"})
MERGE (p)-[:CAUSES]->(m)
"""
run_cypher(query)

[]

In [7]:
# 7.2) Transmission Vector
query = """
MATCH (m:Disease {name:"Malaria"})
MERGE (v:Vector {name:"Anopheles mosquito"})
MERGE (m)-[:TRANSMITTED_BY]->(v)
"""
run_cypher(query)

[]

In [8]:
# 7.3) Symptoms
symptoms = ["Fever", "Chills", "Headache", "Anaemia"]

query = """
UNWIND $symptoms AS symptom
MATCH (m:Disease {name:"Malaria"})
MERGE (s:Symptom {name:symptom})
MERGE (m)-[:HAS_SYMPTOM]->(s)
"""

run_cypher(query, {"symptoms": symptoms})

[]

In [9]:
# 7.4) Prevention Methods
methods = [
    "Insecticide-treated nets",
    "Indoor residual spraying",
    "Environmental sanitation"
]

query = """
UNWIND $methods AS method
MATCH (m:Disease {name:"Malaria"})
MERGE (p:PreventionMethod {name:method})
MERGE (m)-[:PREVENTED_BY]->(p)
"""

run_cypher(query, {"methods": methods})

[]

In [10]:
# 7.5) Treatments
treatments = [
    "Artemisinin-based combination therapy (ACT)",
    "Antimalarial prophylaxis"
]

query = """
UNWIND $treatments AS treatment
MATCH (m:Disease {name:"Malaria"})
MERGE (t:Treatment {name:treatment})
MERGE (m)-[:TREATED_WITH]->(t)
"""

run_cypher(query, {"treatments": treatments})

[]

In [11]:
# 7.6) Vulnerable Population Groups
groups = ["Children under five", "Pregnant women"]

query = """
UNWIND $groups AS group
MATCH (m:Disease {name:"Malaria"})
MERGE (g:PopulationGroup {name:group})
MERGE (m)-[:AFFECTS]->(g)
"""

run_cypher(query, {"groups": groups})

[]

In [ ]:
# Visualise the MalKG in Neo4J by running the following query. 
#MATCH (n)-[r]->(m)
#RETURN n, r, m


<H3> 8) Educational Queries (Run directly in Jupyter) </H3>

In [12]:
# 8.1) What causes malaria?
query = """
MATCH (p:Parasite)-[:CAUSES]->(m:Disease {name:"Malaria"})
RETURN p.name AS Cause
"""
run_cypher(query)


[{'Cause': 'Plasmodium falciparum'}]

In [14]:
# 8.2) How is malaria transmitted?
query = """
MATCH (m:Disease {name:"Malaria"})-[:TRANSMITTED_BY]->(v)
RETURN v.name AS Transmission
"""
run_cypher(query)


[{'Transmission': 'Anopheles mosquito'}]

In [15]:
# 8.3) What are the symptoms of malaria?
query = """
MATCH (m:Disease {name:"Malaria"})-[:HAS_SYMPTOM]->(s)
RETURN s.name AS Symptom
"""
run_cypher(query)


[{'Symptom': 'Fever'},
 {'Symptom': 'Chills'},
 {'Symptom': 'Headache'},
 {'Symptom': 'Anaemia'}]

In [16]:
# 8.4) How can malaria be prevented?
query = """
MATCH (m:Disease {name:"Malaria"})-[:PREVENTED_BY]->(p)
RETURN p.name AS Prevention
"""
run_cypher(query)


[{'Prevention': 'Insecticide-treated nets'},
 {'Prevention': 'Indoor residual spraying'},
 {'Prevention': 'Environmental sanitation'}]

In [17]:
# 8.5) Who is most affected by malaria?
query = """
MATCH (m:Disease {name:"Malaria"})-[:AFFECTS]->(g)
RETURN g.name AS Vulnerable_Group
"""
run_cypher(query)


[{'Vulnerable_Group': 'Children under five'},
 {'Vulnerable_Group': 'Pregnant women'}]

In [16]:
# 9) Visualise the KG in Jupyter Notebook
#Step 1: Install required packages (run once)
#!pip install pyvis

In [18]:
#Step 2: Import libraries
from pyvis.network import Network


In [18]:
#Step 3: Query Neo4j for nodes and relationships
# We will fetch nodes and edges explicitly.
#query = """
#MATCH (n)-[r]->(m)
#RETURN n, r, m
#"""

#records = run_cypher(query)
#records


In [19]:
#Step 3: Query Neo4j for nodes and relationships
# and fetch the nodes and edges explicitly.

query = """
MATCH (n)-[r]->(m)
RETURN n, type(r) AS rel, m
"""
records = run_cypher(query)


In [20]:
# Step 4: Create an interactive network
net = Network(
    height="600px",
    width="100%",
    bgcolor="#ffffff",
    font_color="black",
    directed=True
)

In [21]:
#Step 5: Add nodes and edges to the network
added_nodes = set()

for record in records:
    n = record["n"]
    m = record["m"]
    r = record["rel"]

    n_id = n["name"]
    m_id = m["name"]

    # Add source node
    if n_id not in added_nodes:
        net.add_node(
            n_id,
            label=n_id,
            title=str(n)
        )
        added_nodes.add(n_id)

    # Add target node
    if m_id not in added_nodes:
        net.add_node(
            m_id,
            label=m_id,
            title=str(m)
        )
        added_nodes.add(m_id)

    # Add edge
    net.add_edge(
        n_id,
        m_id,
        label=r
    )

In [22]:
#Step 6: Render inside Jupyter
from IPython.display import HTML, display
net.write_html("malaria_kg.html")

display(HTML("malaria_kg.html"))


<H3> 9) Visualize the KG (Neo4j Browser) </H3>

In Neo4j Browser, run:<br>
MATCH (n)-[r]->(m) <br>
RETURN n, r, m <br>


In [47]:
# 10) Clean shutdown (good practice)
driver.close()
print("Connection closed")


Connection closed
